In [1]:
import pandas as pd
shot = pd.read_csv("./convert_shot.csv")
shot = shot["player"]
count = shot.value_counts()
print(count)

14    3195
33    2774
44    1977
24    1937
45    1862
23    1610
13    1259
12    1151
47     920
18     882
46     832
19     797
39     653
43     548
53     548
42     521
52     516
38     506
60     503
40     463
41     450
55     444
25     435
61     432
51     418
50     392
8      337
15     334
54     311
7      298
56     287
35     264
49     253
3      248
57     246
27     241
29     232
34     228
58     226
59     226
48     223
26     218
4      205
6      189
2      181
28     180
17     171
20     166
16     160
1      158
21     149
5      142
22     130
11     113
Name: player, dtype: int64


In [26]:
import pandas as pd
import numpy as np

# 讀取資料
shot = pd.read_csv("./convert_shot.csv")

# 過濾數據框以獲取特定玩家和擊球次數不等於1的記錄
filtered_shot_df = shot[(shot["player"] == 14) | (shot["player"] == 33)]
filtered_shot_df = filtered_shot_df[filtered_shot_df["shot_num"] != 1]

# 創建'receive_type'列，通過匹配shot_id和前一個shot_id
filtered_shot_df = filtered_shot_df.copy()
filtered_shot_df['receive_type'] = filtered_shot_df['shot_id'].apply(
    lambda x: shot.loc[shot['shot_id'] == x - 1, 'ball_type'].values[0] if len(shot.loc[shot['shot_id'] == x - 1, 'ball_type'].values) > 0 else None
)

# 定義一個函數來計算擊球點與每個球員之間的距離，並標註最近的球員
def find_nearest_player(row):
    if row['player'] == 14:
        if (row['hit_y'] > 67) ^ (row['player_A_y'] > 67):
            return {'nearest_player_id': 1}
        else:
            return {'nearest_player_id': 3}
    if row['player'] == 33:
        if (row['hit_y'] > 67) ^ (row['player_A_y'] > 67):
            return {'nearest_player_id': 2}
        else:
            return {'nearest_player_id': 4}

# 計算每行數據中擊球點與球員的距離，並標註最近的球員
nearest_players = filtered_shot_df.apply(find_nearest_player, axis=1, result_type='expand')
filtered_shot_df = pd.concat([filtered_shot_df, nearest_players], axis=1)

# 保存過濾後的數據框到新的 CSV 文件
filtered_shot_df.to_csv('filtered_shot.csv', index=False, encoding='utf-8')

# 顯示前幾行結果
print(filtered_shot_df.head())


     shot_id  rally_id  shot_num      time  start_frame_num  end_frame_num  \
665      666        73         4  00:06:14            11227          11256   
669      670        73         8  00:06:17            11314          11360   
671      672        73        10  00:06:19            11377          11403   
673      674        73        12  00:06:20            11415          11440   
677      678        74         2  00:06:39            11971          11989   

     player ball_type  backhand  player_A_x  ...  player_D_y      hit_x  \
665      14        長球         0   36.265301  ...   90.150311   3.885840   
669      14        挑球         1   29.682396  ...  102.054657  28.887059   
671      14        挑球         0   24.380195  ...  108.041393   5.564773   
673      14        挑球         1   28.676133  ...  110.086375  22.328306   
677      14       推撲球         0   29.682633  ...  103.408811  34.828785   

          hit_y   return_x   return_y  hit_height  return_height  flaw  \
665  1

In [40]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# set traditional Chinese display
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']

# 讀取資料
data = pd.read_csv("./filtered_shot.csv")

# 將player C和player D作為player A和player B的位置（如果最近的球員ID大於2）
mask = data['nearest_player_id'] > 2
data.loc[mask, ['player_A_x', 'player_A_y', 'player_B_x', 'player_B_y']] = data.loc[mask, ['player_C_x', 'player_C_y', 'player_D_x', 'player_D_y']].values

# 定義一個函數來排序座標
def sort_coordinates(row):
    coords = sorted([(row['player_A_x'], row['player_A_y']), (row['player_B_x'], row['player_B_y'])])
    return pd.Series({'player_A_x': coords[0][0], 'player_A_y': coords[0][1], 'player_B_x': coords[1][0], 'player_B_y': coords[1][1]})

# 定義一個函數來對稱變換 y 座標
def reflect_y(row):
    if row['player_A_y'] > 67:
        row['player_A_y'] = 134 - row['player_A_y']  # 對稱變換
        # row['player_A_x'] = 61 - row['player_A_x']
    if row['player_B_y'] > 67:
        row['player_B_y'] = 134 - row['player_B_y']  # 對稱變換
        # row['player_B_x'] = 61 - row['player_B_x']
    return row

# 應用排序函數來處理每一行數據
sorted_coords = data.apply(sort_coordinates, axis=1)
data[['player_A_x', 'player_A_y', 'player_B_x', 'player_B_y']] = sorted_coords

# 應用對稱變換函數來處理每一行數據
data = data.apply(reflect_y, axis=1)

# 選取我們感興趣的特徵，包括兩個球員的 x 和 y 座標
features = data[['player_A_x', 'player_A_y', 'player_B_x', 'player_B_y']]

# 使用 KMeans 進行聚類
n_clusters = 6  # 假設我們要分成 6 類
kmeans = KMeans(n_clusters=n_clusters)
data['cluster'] = kmeans.fit_predict(features)

# 繪圖顯示聚類結果並存儲圖片
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for cluster in range(n_clusters):
    plt.figure(figsize=(9.15, 10.05))
    cluster_data = data[data['cluster'] == cluster]
    plt.scatter(cluster_data['player_A_x'], cluster_data['player_A_y'], color='r', label=f'Cluster {cluster} - Player A')
    plt.scatter(cluster_data['player_B_x'], cluster_data['player_B_y'], color='b', label=f'Cluster {cluster} - Player B')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.xlim(0, 61)  # 設置 x 軸範圍
    plt.ylim(0, 67)  # 設置 y 軸範圍
    plt.title(f'Cluster {cluster} Player Positions')
    plt.savefig(f'cluster_{cluster}_positions.jpg')
    plt.close()
    
plt.figure(figsize=(9.15, 10.05))
for cluster in range(n_clusters):
    cluster_data = data[data['cluster'] == cluster]
    plt.scatter(cluster_data['player_A_x'], cluster_data['player_A_y'], color=colors[cluster % len(colors)], label=f'Cluster {cluster} - Player A')
    plt.scatter(cluster_data['player_B_x'], cluster_data['player_B_y'], color=colors[cluster % len(colors)], marker='x', label=f'Cluster {cluster} - Player B')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.xlim(0, 61)  # 設置 x 軸範圍
plt.ylim(0, 67)  # 設置 y 軸範圍
plt.legend()
plt.title(f'Clusters Player Positions')
plt.savefig(f'clusters_positions.jpg')
plt.close()
# 統計每個聚類中不同球種的數量
cluster_ball_type_counts = data.groupby(['cluster', 'ball_type']).size().unstack(fill_value=0)

# 將聚類結果和統計結果存儲到新的 CSV 文件
data.to_csv('clustered_shot.csv', index=False, encoding='utf-8')
cluster_ball_type_counts.to_csv('cluster_ball_type_counts.csv', encoding='utf-8')

# 顯示統計結果
print("Cluster ball type counts sorted by frequency:")
sorted_cluster_ball_type_counts = cluster_ball_type_counts.apply(lambda x: x.sort_values(ascending=False), axis=1)
print(sorted_cluster_ball_type_counts)

# 為每個聚類畫出球種長條圖並存儲圖片
for cluster in range(n_clusters):
    try:
        plt.figure(figsize=(10, 6))
        sorted_counts = sorted_cluster_ball_type_counts.loc[cluster]
        sorted_counts.plot(kind='bar', color='skyblue')
        plt.title(f'Cluster {cluster} Ball Type Distribution')
        plt.xlabel('Ball Type')
        plt.ylabel('Frequency')
        plt.ylim(0, 850)  # 設置 y 軸範圍

        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'cluster_{cluster}_ball_type_distribution.png')
        plt.close()
    except:
        continue

Cluster ball type counts sorted by frequency:
          切球  勾球  小平球  平球  後場抽平球   挑球  推撲球  推球  撲球  擋小球  放小球   殺球  網前小球  過度切球  \
cluster                                                                         
0          5   5   35  36      1  406   32  67  35   57   97   41     5     5   
1        180   0    4  20      6   27    3   3  11   58   19  466     4     6   
3        187   2    1  22      4   34    3   4  17   60   11  417     2     7   
4          5   5   44  54      2  432   25  70  48   59   84   34     8     0   
5         18   3   13  77      2  186    1  15   1  186    8   63    34     1   

          長球  防守回抽  防守回挑  
cluster                   
0         28    10    21  
1        158     1    11  
3        174     2    20  
4         39     8    25  
5         63   395   809  


<Figure size 720x432 with 0 Axes>